In [1]:
import tensorflow as tf
import tangent

import numpy as np
import numpy
tf.enable_eager_execution()

/home/act65/anaconda3/envs/dynamics/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
n = 8
batch_size = 50
x = tf.random_normal([batch_size, n])
W = tf.random_normal([n,n])

In [3]:
def energy_fn(x, W):
    return tf.reduce_mean(-0.5*tf.reduce_sum(tf.matmul(x, tf.matmul(W, x,transpose_b=True)), axis=1))

print(energy_fn(x, W).shape)

dEdx = tangent.autodiff(energy_fn)

print(dEdx(x, W, b_return=tf.ones([])).shape)

()
(50, 8)


In [4]:
def step(x, W):
    grad = dEdx(x, W, b_return=tf.constant(1, dtype=tf.float32))
    return x - 0.1 * grad

print(step(x, W).shape)


(50, 8)


In [5]:

dstepdx = tangent.autodiff(step, mode='forward')
dstepdx(x, W, dx=tf.ones_like(x)).shape

AttributeError: 

In [13]:
forward_dfdx = tangent.autodiff(step, mode='forward', wrt=(2,))
forward_dfdx(x, tf.ones_like(x), W, dW=tf.ones_like(W)).shape

TensorShape([Dimension(50), Dimension(8)])

In [13]:
def forward(state, n_steps=1):
    
    def _step(i, state, dydx):
        # a wrapper for self.step(...)
        new_state = step(state, W)
        return i + 1, new_state, forward_dfdx(x, W, dx=dydx)

    with tf.name_scope('forward'):
        while_condition = lambda i, j, k : tf.less(i, n_steps)
        i = tf.constant(0)
        dydx = tf.ones_like(state)
        new_i_, new_state, new_dydx = tf.while_loop(
            while_condition, 
            _step, 
            loop_vars=[i, state, dydx], 
            back_prop=False)

        return new_state, new_dydx

In [14]:
y, dy = forward(x)

TypeError: step() missing 1 required positional argument: 'W'

In [38]:
dy.shape

TensorShape([Dimension(50), Dimension(2)])

In [64]:
import tangent
from tangent.grads import adjoint

def cube(x):
    return x * x * x
  
# Register the gradient of cube with Tangent
# NOTE! This is not a runnable function, but instead is a code template.
# Tangent will replace the names of the variables `result` and `x` with whatever
# is used in your containing function.
@adjoint(cube)
def dcube(result, x):
    d[x] = d[result] * 3 * x * x

def f(val):
    cubed_val = cube(val)
    return cubed_val

print(tangent.grad(f,verbose=1))

def dfdval(val, bcubed_val=1.0):
    cubed_val = cube(val)
    assert tangent.shapes_match(cubed_val, bcubed_val
        ), 'Shape mismatch between return value (%s) and seed derivative (%s)' % (
        numpy.shape(cubed_val), numpy.shape(bcubed_val))

    # Grad of: cubed_val = cube(val)
    _bval = bcubed_val * 3 * val * val
    bval = _bval
    return bval

<function dfdval at 0x7f0b647340d0>
